In [31]:
import time
import random
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
import requests
from lxml import html
import csv
from redis import StrictRedis
import json

In [32]:
# 连接缓存数据库
redis = StrictRedis(host='127.0.0.1', port=6379)
# 1岗位搜索列表页面链接列表
links_list = list()
# 2岗位详情列表页面链接队列
job_links_queue = Queue()
# 3待保存数据队列
job_info_queue = Queue()
# 每次开始抓取清空缓存
redis.delete('crawled_links')

0

In [33]:
cookie = 'guid=a07ece7b2525a88b80265c4041ebc65c; nsearch=jobarea%3D%26%7C%26ord_field%3D%26%7C%26recentSearch0%3D%26%7C%26recentSearch1%3D%26%7C%26recentSearch2%3D%26%7C%26recentSearch3%3D%26%7C%26recentSearch4%3D%26%7C%26collapse_expansion%3D; search=jobarea%7E%60030200%7C%21ord_field%7E%600%7C%21'
headers = {
    'Host':
        'search.51job.com',
    'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
    'Accept':
        'application/json, text/javascript, */*; q=0.01',
    'Referer':
        'https://search.51job.com/list/030200,000000,0000,00,9,99,python,2,2.html?lang=c&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&ord_field=0&dibiaoid=0&line=&welfare=',
    'Cookie':
        cookie,
    'X-Requested-With':
        'XMLHttpRequest'
}
start_url = 'https://search.51job.com/list/030200,000000,0000,00,9,99,python,2,1.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare='

keyword = 'python'
pages = 1
csv_writer = csv.writer(open(f'{keyword}_jobs.csv', 'a', encoding='utf-8'))
for i in range(1, pages + 1):
    url = f'https://search.51job.com/list/030200,000000,0000,00,9,99,{keyword},2,{i}.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare='
    links_list.append(url)

In [34]:
# 获取页面对象
def get_page(url, headers):
    time.sleep(random.randint(1, 3))
    try:
        res = requests.get(url=url, headers=headers)
        page = res.content.decode('gbk')
    except Exception as e:
        print(e)
        return None
    else:
        # 已经爬取的链接加入到redis记录
        redis.sadd('crawed_links', url)
    try:
        dom = html.document_fromstring(page)
    except Exception as e:
        print(e)
        return None
    return dom

In [35]:
def extract_list_links():
    global headers
    for url in links_list:
        # 请求链接
        req = requests.get(url, timeout=5, headers=headers)
        # 获取内容，gbk解码
        data = req.content.decode('gbk')
        print('success loading', url)
        # 将json内容数据解析
        try:
            jobs = json.loads(data)
        except Exception as e:
            print(e)
        else:
            # 找到每一页的链接
            for i in range(len(jobs['engine_search_result'])):
                job_link = jobs['engine_search_result'][i]['job_href']
                job_links_queue.put(job_link)
            print('job_links_len:', job_links_queue.qsize())

In [36]:
extract_list_links()

success loading https://search.51job.com/list/030200,000000,0000,00,9,99,python,2,1.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
job_links_len: 50
success loading https://search.51job.com/list/030200,000000,0000,00,9,99,python,2,2.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
job_links_len: 100
success loading https://search.51job.com/list/030200,000000,0000,00,9,99,python,2,3.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
job_links_len: 150
su

In [37]:
# # 从页面提取有用信息
def extract_info():
    c=0
    while c<20:
        c+=1
        try:
            refer = redis.srandmember('crawed_links', 1)[0].decode()
        except Exception as e:
            print(e)
            refer = start_url
        # 增加headers反爬
        global headers
        headers['Referer'] = refer
        # 各种信息的XPATH语法
        job_name_xpath = "//h1/@title"
        job_salary_xpath = "//div[@class='cn']/strong/text()"
        job_info_xpath = "//p[@class='msg ltype']"
        job_stars_xpath = "//div[@class='t1']/span[@class='sp4']/text()"  # list
        job_description_xpath = "//div[@class='bmsg job_msg inbox']/p/text()"  # list
        job_category_xpath = "//div[@class='bmsg job_msg inbox']/div/p[1]/a/text()"  # list
        job_contact_xpath = "//div[@class='bmsg inbox']/p/text()"
        company_introduction_xpath = "//div[@class='tmsg inbox']"
        company_name_xpath = "//div[@class='com_msg']"
        company_type_xpath = "//div[@class='com_tag']/p[1]"
        company_size_xpath = "//div[@class='com_tag']/p[2]"
        company_industry_xpath = "//div[@class='com_tag']/p[3]"
        company_url_xpath = "//p[@class='cname']/a[@class='catn']/@href"
        # 将网页变成dom对象
        try:
            job_link = job_links_queue.get(block=False)
            print('job_link:', job_link)
        except Exception as e:
            print(e)
        else:
            dom = get_page(job_link, headers=headers)
            if dom is None:
                return False
        # 用lxml解析dom对象
        try:
            job_name = dom.xpath(job_name_xpath)[0]
            job_salary = dom.xpath(job_salary_xpath)[0]
            job_info = dom.xpath(job_info_xpath)[0].text_content()
            job_stars = dom.xpath(job_stars_xpath)
            job_description = dom.xpath(job_description_xpath)
            job_category = dom.xpath(job_category_xpath)
            job_contact = dom.xpath(job_contact_xpath)[0]
            company_introduction = dom.xpath(
                company_introduction_xpath)[0].text_content().strip()
            company_name = dom.xpath(
                company_name_xpath)[0].text_content().strip()
            company_type = dom.xpath(company_type_xpath)[0].text_content()
            company_size = dom.xpath(company_size_xpath)[0].text_content()
            company_industry = dom.xpath(
                company_industry_xpath)[0].text_content().strip()
            company_url = dom.xpath(company_url_xpath)[0]
        except Exception as e:
            print(e, 'missing some values')
        else:
            print([job_name, job_salary, job_info])
            job_stars = ','.join(job_stars)
            job_description = ','.join(job_description)
            job_category = ','.join(job_category)
            job_info_queue.put([
                job_name, job_salary, job_info, job_stars, job_category,
                job_description, job_contact, company_name, company_type,
                company_size, company_industry, company_introduction, job_link,
                company_url
            ])

In [38]:
extract_info()

job_link: https://jobs.51job.com/guangzhou/124980175.html?s=01&t=0
['Python开发工程师', '4.5-8千/月', '广州\xa0\xa0|\xa0\xa01年经验\xa0\xa0|\xa0\xa0大专\xa0\xa0|\xa0\xa0招3人\xa0\xa0|\xa0\xa008-30发布']
job_link: https://jobs.51job.com/guangzhou/124416677.html?s=01&t=0
list index out of range missing some values
job_link: https://jobs.51job.com/guangzhou-hpq/123273059.html?s=01&t=0
['Python开发工程师', '0.6-1.2万/月', '广州-黄埔区\xa0\xa0|\xa0\xa03-4年经验\xa0\xa0|\xa0\xa0本科\xa0\xa0|\xa0\xa0招1人\xa0\xa0|\xa0\xa008-30发布']
job_link: https://jobs.51job.com/guangzhou/124283855.html?s=01&t=0
['数据库财务实习生（Python工程师）2329 (职位编号：Meixin002329)', '2-3千/月', '广州\xa0\xa0|\xa0\xa0在校生/应届生\xa0\xa0|\xa0\xa0大专\xa0\xa0|\xa0\xa0招1人\xa0\xa0|\xa0\xa008-30发布']
job_link: https://jobs.51job.com/guangzhou/124955221.html?s=01&t=0
['Python开发工程师 (职位编号：UR002262)', '1.3-1.8万/月', '广州\xa0\xa0|\xa0\xa03-4年经验\xa0\xa0|\xa0\xa0本科\xa0\xa0|\xa0\xa0招若干人\xa0\xa0|\xa0\xa008-30发布']
job_link: https://jobs.51job.com/guangzhou-thq/124253821.html?s=01&t=0
['Python开发工程

In [39]:
def save_info():
    while True:
        try:
            job_info = job_info_queue.get(block=False)
        except Exception as e:
            print(e)
        else:
            csv_writer.writerow(job_info)

In [ ]:
save_info()